In [1]:
import cPickle, os
import pdb
import numpy as np

def load_pickle(pickle_path):
    with open(pickle_path, 'rb') as fo:
        pickle_dict = cPickle.load(fo)
    return pickle_dict  

def combine_pickles(sample_names_pickle, REA_pickle, SRF_pickle, PED_pickle):
    sample_names = load_pickle(sample_names_pickle)
    REA_predictions = load_pickle(REA_pickle)
    SRF_predictions = load_pickle(SRF_pickle)
    PED_predictions = load_pickle(PED_pickle)
    
    combined_predictions = []
    for i in range(len(REA_predictions)):
        np.place(REA_predictions[i], REA_predictions[i]==2, 1) # first step, replace the SRF and PED to REA
        np.place(REA_predictions[i], REA_predictions[i]==3, 1)
        SRF_locs = (SRF_predictions[i] == 2)
        PED_locs = (PED_predictions[i] == 3)
        
        np.place(REA_predictions[i], SRF_locs, 2)
        np.place(REA_predictions[i], PED_locs, 3)
        
        combined_predictions.append(REA_predictions[i])
        
    return sample_names, REA_predictions

def write_disk(target_root, sample_names, sample_predictions):
    for i in range(len(sample_names)):
        target_path = os.path.join(target_root, sample_names[i]+"_volumes.npy")
        np.save(target_path, sample_predictions[i].astype("uint8")) # save as uint8 to save space

In [3]:
sample_names_pickle = "./predictions/pickles/combination/sample_names_0.7484.pkl"
REA_pickle = "./predictions/pickles/combination/segmentation_sample_predictions_epoch12.pkl"
SRF_pickle = "./predictions/pickles/combination/ensemble_segmentation_sample_predictions_SRF0.3003.pkl"
PED_pickle = "./predictions/pickles/combination/segmentation_sample_predictions_0.7484.pkl"

sample_names, predictions = combine_pickles(sample_names_pickle, REA_pickle, SRF_pickle, PED_pickle)

In [4]:
write_disk("./predictions/segmentation/20181007/", sample_names, predictions)